In [ ]:
# google colab does not come with torch installed. And also, in course we are using torch 0.4. 
# so following snippet of code installs the relevant version

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

  ERROR: HTTP error 403 while getting http://download.pytorch.org/whl/cu101/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
  ERROR: Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu101/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu101/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
ERROR: Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu101/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of HTTP error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu101/torch-0.4.1-cp36-cp36m-linux_x86_64.whl for URL http://download.pytorch.org/whl/cu101/torch-0.4.1-cp36-cp36m-linux_x86_64.whl


In [ ]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
# 
# if it prints otherwise, then you need to enable GPU: 
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [ ]:
# we need pillow version of 5.3.0
# we will uninstall the older version first
!pip uninstall -y Pillow
# install the new one
!pip install Pillow==5.3.0
# import the new one
import PIL
print(PIL.__version__)
# this should print 5.3.0. If it doesn't, then restart your runtime:
# Menu > Runtime > Restart Runtime

Uninstalling Pillow-5.3.0:
  Successfully uninstalled Pillow-5.3.0
  Using cached https://files.pythonhosted.org/packages/62/94/5430ebaa83f91cc7a9f687ff5238e26164a779cca2ef9903232268b0a318/Pillow-5.3.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


5.3.0


In [ ]:
print(torch.__version__)

1.5.1+cu101


In [ ]:
!pip install torchvision

In [ ]:
# Imports here
import torch
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch import optim
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import lr_scheduler

In [ ]:
!gdown https://drive.google.com/uc?id=1ZKNpvb1xZGCwqaY4NfykpeEZnfjx3abQ
!tar -xvf plantdisease.zip
!unzip -qq plantdisease.zip

Downloading...
From: https://drive.google.com/uc?id=1ZKNpvb1xZGCwqaY4NfykpeEZnfjx3abQ
To: /content/plantdisease.zip
343MB [00:01, 239MB/s]
tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors
replace train/Pepper__bell___Bacterial_spot/0022d6b7-d47c-4ee2-ae9a-392a53f48647___JR_B.Spot 8964.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
#data = 'plantdisease'
train = '/content/train'
valid = '/content/valid'

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# TODO: Define your transforms for the training and validation sets

transform = transforms.Compose([transforms.RandomRotation(45),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train, transform=transform)
valid_data = datasets.ImageFolder(valid, transform=transform)

batch_size = 32
# TODO: Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size)

In [ ]:
!wget -cq https://github.com/viritaromero/Plant-diseases-classifier/raw/master/categories.json

In [ ]:
import json
with open('categories.json', 'r') as f:
    cat_to_name = json.load(f)


In [ ]:
# TODO: Build and train your network
model = models.resnet152(pretrained=True)

for param in model.parameters():
    param.requires_grad_(True)

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
classifier =  nn.Sequential(nn.Linear(2048,1024),
                           nn.ReLU(),
                           nn.Dropout(p=0.2),
                           nn.Linear(1024,102),
                           nn.LogSoftmax(dim=1))


model.fc = classifier

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
criterion = nn.NLLLoss()
#criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.fc.parameters(), lr = 0.001)
#optimizer = optim.SGD(model.fc.parameters(), lr = 0.0001)

#scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)



device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

print(device)

cuda


In [ ]:
epochs = 50
train_losses, valid_losses = [], []
valid_loss_min = np.Inf
for e in range(epochs):
    print('Epoch: {}/{}'.format(e + 1, epochs))
    running_loss = 0
    valid_loss = 0
    #scheduler.step()
    model.train()
  
    with torch.set_grad_enabled(True):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            log_ps = model(images)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            
            
     
    with torch.no_grad():
        accuracy = 0
        model.eval()
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            
            log_ps = model(images)
            loss = criterion(log_ps, labels)
            
            valid_loss += loss.item() * images.size(0)
            
            ps = torch.exp(log_ps)
            
            _, top_class = ps.topk(1, dim = 1)
            
            equals = top_class == labels.view(*top_class.shape)
            
            accuracy +=  torch.sum(equals.type(torch.FloatTensor))
            
    train_losses.append(running_loss/len(train_data))
    valid_losses.append(valid_loss/len(valid_data))
    
    print('Training loss: {:.6f} ..'.format(running_loss/len(train_data)),
          'Validation loss: {:.6f} ..'.format(valid_loss/len(valid_data)),
          'Training Accuracy: {:.3f}'.format(accuracy/len(train_data)),
          'Validation Accuracy: {:.3f}'.format(accuracy/len(valid_data))
          )   

Epoch: 1/50
Training loss: 1.002568 .. Validation loss: 0.616715 .. Training Accuracy: 0.358 Validation Accuracy: 0.794
Epoch: 2/50
Training loss: 0.638034 .. Validation loss: 0.505752 .. Training Accuracy: 0.377 Validation Accuracy: 0.837
Epoch: 3/50
Training loss: 0.579950 .. Validation loss: 0.496748 .. Training Accuracy: 0.377 Validation Accuracy: 0.836
Epoch: 4/50
Training loss: 0.547151 .. Validation loss: 0.411576 .. Training Accuracy: 0.387 Validation Accuracy: 0.859
Epoch: 5/50
Training loss: 0.512136 .. Validation loss: 0.393962 .. Training Accuracy: 0.390 Validation Accuracy: 0.865
Epoch: 6/50
Training loss: 0.483908 .. Validation loss: 0.373346 .. Training Accuracy: 0.395 Validation Accuracy: 0.877
Epoch: 7/50
Training loss: 0.481556 .. Validation loss: 0.379889 .. Training Accuracy: 0.393 Validation Accuracy: 0.873
Epoch: 8/50
Training loss: 0.474780 .. Validation loss: 0.334543 .. Training Accuracy: 0.401 Validation Accuracy: 0.890
Epoch: 9/50
Training loss: 0.427283 .. V